## install Apache spark
#### https://spark.apache.org/downloads.html

### Starting spark using : spark-shell

and then we start processing our data using pyspark

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder.appName("BatchProcessing").getOrCreate()

# Load data from the local JSON file into a DataFrame
df = spark.read.option("multiline","true").json(r"comments_analysis.json")

# Perform batch processing to count the number of comments per author
result = df.groupBy("author").count().collect()

# Show result
for row in result:
    print(f"Author: {row[0]}\nNumber of comments: {row[1]}\n")

# Stop Spark session
spark.stop()

Author: مواطن 
Number of comments: 4

Author: كمال بيه 
Number of comments: 1

Author: الاحتلال أنواع 
Number of comments: 1

Author: ahmed 
Number of comments: 4

Author: ابوجهل 
Number of comments: 1

Author: احمد 
Number of comments: 1

Author: خريبگي 
Number of comments: 1

Author: عكاشة 
Number of comments: 1

Author: العريشي 
Number of comments: 1

Author: نواب اخر زمان 
Number of comments: 1

Author: مواطن معاق 
Number of comments: 1

Author: Me again 
Number of comments: 1

Author: عبد الوافي. 
Number of comments: 1

Author: السيّارين والسيّارات.. 
Number of comments: 1

Author: عينك ميزانك 
Number of comments: 1

Author: زهوة 
Number of comments: 1

Author: المعطل 
Number of comments: 1

Author: Warzazi 
Number of comments: 1

Author: مهاجر من الشمال 
Number of comments: 1

Author: Kamal 
Number of comments: 1

Author: فقط القضاء على السيبة 
Number of comments: 1

Author: محمد 
Number of comments: 1

Author: السي محمد 
Number of comments: 1

Author: Berhoc 
Number of comments:

In [5]:
from operator import add
import re
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder.appName("Batch Processing").getOrCreate()

# Load data from the local JSON file into a DataFrame
df = spark.read.option("multiline","true").json(r"comments_analysis.json")

# Create an RDD from the "text" column
text_rdd = df.rdd.flatMap(lambda x: re.split("\W+", x.text))

# Count the frequency of each word
word_counts = text_rdd.map(lambda word: (word, 1)).reduceByKey(add)

# Sort the words by frequency
sorted_word_counts = word_counts.sortBy(lambda x: x[1], ascending=False)

# Show the top 10 most used words
top_10 = sorted_word_counts.take(10)
for word, count in top_10:
    print(f"{word}: {count}")

# Stop Spark session
spark.stop()

: 122
في: 80
و: 73
من: 71
على: 41
لا: 36
ان: 23
او: 22
هو: 21
أن: 20


## Sentiment Analysis

In [9]:
import pandas as pd
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier

# Load the data into a pandas dataframe
df = pd.read_csv('./../Data/comments_politique.csv')

# Show the first few rows of the dataframe
df.head()

,Unnamed: 0,postId,comment,score,topic
0,0,3a7f2f5e04dd11eba84c646e69d991ea,فليجتمعوا لما فيه خير ليبيا و ينهو هذا الصراع ...,69,politique
1,1,3a7f2f5e04dd11eba84c646e69d991ea,ايوى الرؤساء لن يحضروا يعني هناك ضغوط خارجية و...,-18,politique
2,2,3a7f2f5e04dd11eba84c646e69d991ea,ا لامبراطورية المغربية العظمى لها تاريخ وشعب ع...,29,politique
3,3,3a7f2f5e04dd11eba84c646e69d991ea,الحل الوحيد هو تقسيم ليبيا الي الشرق والغرب هذ...,-87,politique
4,4,3a7f2f5e04dd11eba84c646e69d991ea,نتمنى لاخوة اللبيين ان يتوافقوا و يتفقوا على م...,27,politique


In [10]:
df['score']

0        69
1       -18
2        29
3       -87
4        27
         ..
45852    -4
45853     3
45854     3
45855     3
45856     1
Name: score, Length: 45857, dtype: int64

In [13]:
# Select a smaller subset of the data
subset = df.sample(frac=0.1)

# Preprocess the text data
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(subset['comment'])

# Train the model
model = DecisionTreeClassifier()
model.fit(features, subset['score'])

# Load the scraped data into a pandas dataframe
scraped_data = pd.read_json("./../Data/comments.json")

# Extract the text field from the scraped data
scraped_comments = scraped_data['text'].tolist()

# Preprocess the scraped comments
scraped_features = vectorizer.transform(scraped_comments)

# Make predictions on the scraped comments
scraped_scores = model.predict(scraped_features)
print(scraped_scores)

[   3    1    4    2   18   30  245    1    6    2    2   10    4   17
    1   73    1   41    6   18 -323   18    3    3    3   28    0   11
    0    4 -118    0   -5   73    0  342  156    8   18    2    2   23
    2    5   32   27    4    7   73    4    8    9   36    3   -2    4
    2    1   63    3    0]


In [14]:
# Evaluate the accuracy of the model
accuracy = model.score(features, subset['score'])
print("Accuracy: ", accuracy)

Accuracy:  0.9947666812036633


In [15]:
# Create a new dataframe from the scraped comments and their scores
scraped_df = pd.DataFrame({'comments': scraped_comments, 'scores': scraped_scores})

# Iterate over the rows of the dataframe and print the comment and its score
for index, row in scraped_df.iterrows():
    print("Comment: ", row['comments'])
    print("Score: ", row['scores'])
    print("\n")
scraped_df

Comment:  
عاقبوا غير أمينكم العام عاد أجيو شوفو الباقي…اللهم اضرب الظالمين بالظالمين و أخرجنا من بينهم سالمين

Score:  3


Comment:  
الفراشة سرطان صامت يقتل المجتمع ببطء

Score:  1


Comment:  
هذا يعطي شرعية لاحتلال الملك العمومي، لتفادي هذا النوع من الحوادث يجب إيقاف كل أشكال احتلال الملك العمومي و التعبئة من أجل هذا الهدف، و ليس الإعتراف بالهزيمة بمثل هاته النصوص.

Score:  4


Comment:  
لكل فعل رد فعل
ولكل نتيجة سبب
والتركيز يكون على السبب لا النتائج
ويكون على النتائج لا على الآراء
بمعنى ان تقديم مقترح قوانين تمس مباشرة بفئة عريضة من المواطنين هو تكريس لدكتاتورية قبة البرلمان في ظل التمتع بالأغلبية واقصاء تام وشامل لمفهوم المعارضة
وبذلك نجد أن النواب لا يحفظون حتى اية كاملة من القرآن
وتجدهم يجدون ويجتهدون في تقديم وتنزيل وتشريع القوانين دون علم او معرفة وانما بدافع الحفاظ على مكتسباتهم المادية وتنفيذا الاوامر
والسؤال المطروح
ماهو البديل

Score:  2


Comment:  
نعم يجب معاقبة محتلي الملك العام لا اصحاب العربات او الدراجات هذا هو صلب العدالة يحيا العدل يحيا القانون 

Score:  18


C

,comments,scores
0,\nعاقبوا غير أمينكم العام عاد أجيو شوفو الباقي...,3
1,\nالفراشة سرطان صامت يقتل المجتمع ببطء\n,1
2,\nهذا يعطي شرعية لاحتلال الملك العمومي، لتفادي...,4
3,\nلكل فعل رد فعل\nولكل نتيجة سبب\nوالتركيز يكو...,2
4,\nنعم يجب معاقبة محتلي الملك العام لا اصحاب ال...,18
...,...,...
56,\nLe problème de la corruption et de l’ignoran...,2
57,\n((عندما يتم احتلال الرصيف يضطر الراجلون إلى ...,1
58,\ndans les démocratie sa on appelle l’obstruct...,63
59,\nعندي الحل:\nعندي حل فعال و جذري لظاهرة الباع...,3


In [16]:
scraped_df['scores']

0      3
1      1
2      4
3      2
4     18
      ..
56     2
57     1
58    63
59     3
60     0
Name: scores, Length: 61, dtype: int64

In [17]:
#Create a sentiment column
sentiments = []
for score in scraped_scores:
    if score > 0:
        sentiments.append("positive")
    elif score < 0:
        sentiments.append("negative")
    else:
        sentiments.append("neutral")

#Add the sentiment column to the scraped_data dataframe
scraped_data['sentiment'] = sentiments

In [18]:
scraped_data

,author,date,text,sentiment
0,نورالدين,السبت 4 فبراير 2023 - 12:04,\nعاقبوا غير أمينكم العام عاد أجيو شوفو الباقي...,positive
1,bidaoui,السبت 4 فبراير 2023 - 12:06,\nالفراشة سرطان صامت يقتل المجتمع ببطء\n,positive
2,Kamal,السبت 4 فبراير 2023 - 12:06,\nهذا يعطي شرعية لاحتلال الملك العمومي، لتفادي...,positive
3,عبدووو,السبت 4 فبراير 2023 - 12:08,\nلكل فعل رد فعل\nولكل نتيجة سبب\nوالتركيز يكو...,positive
4,مهاجر من الشمال,السبت 4 فبراير 2023 - 12:09,\nنعم يجب معاقبة محتلي الملك العام لا اصحاب ال...,positive
...,...,...,...,...
56,ahmed,السبت 4 فبراير 2023 - 20:41,\nLe problème de la corruption et de l’ignoran...,positive
57,السيّارين والسيّارات..,السبت 4 فبراير 2023 - 21:12,\n((عندما يتم احتلال الرصيف يضطر الراجلون إلى ...,positive
58,ahmed,السبت 4 فبراير 2023 - 22:09,\ndans les démocratie sa on appelle l’obstruct...,positive
59,Warzazi,السبت 4 فبراير 2023 - 23:03,\nعندي الحل:\nعندي حل فعال و جذري لظاهرة الباع...,positive
